In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain_groq import ChatGroq
from langchain_community.llms import OpenAIChat
from langchain.graphs import Neo4jGraph

In [ ]:
import neo4j
from langchain_groq import ChatGroq
graph=Neo4jGraph(NEO4J_URI,NEO4J_USERNAME,NEO4J_PASSWORD)

In [ ]:
llm = ChatGroq(temperature=0, groq_api_key=groq_api, model_name="mixtral-8x7b-32768")

In [ ]:
graph.schema

'Node properties are the following:\nPatient {id: STRING, name: STRING, gender: STRING, age: INTEGER, location: STRING},Prescription {id: STRING, end_date: STRING, instructions: STRING, dosage: STRING, medication: STRING, time: STRING, start_date: STRING, frequency: STRING}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Patient)-[:HAS_PRESCRIPTION]->(:Prescription),(:Prescription)-[:FOR_PATIENT]->(:Patient)'

In [ ]:
# p1= "Consider yourself as a CYPHER language expert for Neo4j. Based on the below, generate the query. Do not include any comments or explanations"
# p2 = GraphCypherQAChain.from_llm(
#     ChatGroq(temperature=0, groq_api_key=groq_api, model_name="mixtral-8x7b-32768"), graph=graph, verbose=True)

p2 = GraphCypherQAChain.from_llm(
    OpenAIChat(temperature=0,), graph=graph, verbose=True)

langchain.chains.graph_qa.cypher.GraphCypherQAChain

# **Start Here**

In [72]:
# %pip install groq
# %pip install neo4j
# %pip install langchain
# !pip install openai

In [2]:
from groq import Groq
import neo4j
from langchain.graphs import Neo4jGraph

In [75]:
def first_ans(question, response):
  print(response)
  query=f'''
  Consider you are answering a patient over a phone call
  Following is the Question asked by Patient: {question}
  Following is the answer for the question: {response}.

  Use this information to make remind the person in the question and answer any queries they may have to take his medication. mention the time as well.
  Example reminder: Hi XYZ. This is to remind you that you have to take your abc medication at x o clock. Have you taken it?
  I only want the response. Stricitly no comments or explantion of any type
  '''
  ans=call_llm(query)
  return ans

In [77]:
def conversation(bot):
  query='Consider you are a medicine assistant reminder and call agent serving medical queries over phone. Complete the follwing conversation playing the role of medicine reminder assistant. If user has any queries, inform them that you will pass it on to the doctor.\n\n I only want the response. Stricitly no comments, no notes or explantion of any type'
  user_msg= input()
  while True:
    if user_msg:
      user_msg='Patient: '+user_msg
      bot_msg= 'Medicine assistant reminder: '+ bot
      # print(user_msg)
      query= query+'\n'+ bot_msg+'\n'+user_msg
      print(query)
      bot=call_llm(query)
      print("Bot: "+bot)
      user_msg= input()
    else:
      break


In [59]:
def load_graph(username,password,uri):
  # graph=(NEO4J_URI,NEO4J_USERNAME,NEO4J_PASSWORD)
  graph= Neo4jGraph(uri,username,password)
  return graph

In [78]:
def return_query(graph):
  query="Consider yourself as a Neo4J expert. Given the following schema of a Neo4j graph database, "+ str(graph.schema)+" \nWrite a cypher query to answer the following question: \n. "
  question=input()
  query=query+question
  query=query+" Just give the query so that I can copy paste the query and directly execute it. Stricitly no comments or special characters or explantion of any type"
  response=call_llm(query)
  response=response.replace("`",'')
  return question,graph.query(response)


In [79]:


graph=load_graph(NEO4J_USERNAME,NEO4J_PASSWORD,NEO4J_URI)
response=return_query(graph)
bot1_ans=(first_ans(response[0],response[1]))
print(bot1_ans)
conversation(bot1_ans)

Jane Smith
[{'prescription': {'end_date': '2024-04-15', 'instructions': 'Take on an empty stomach', 'dosage': '20mg', 'medication': 'MedB', 'id': 'Rx2', 'time': 'Evening', 'start_date': '2024-03-15', 'frequency': 'Twice daily'}}]
Hello Jane, this is a reminder for your medication. You should take MedB, 20mg, twice daily - once in the morning and once in the evening. The evening dose should be taken on an empty stomach. Today's evening dose is due, have you taken it?
Thanks. I'll take it now. I also feel drowsy after taking medicine
Consider you are a medicine assistant reminder and call agent serving medical queries over phone. Complete the follwing conversation playing the role of medicine reminder assistant. If user has any queries, inform them that you will pass it on to the doctor.

 I only want the response. Stricitly no comments, no notes or explantion of any type
Medicine assistant reminder: Hello Jane, this is a reminder for your medication. You should take MedB, 20mg, twice da